In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-machine-learning-from-disaster/train.csv
/kaggle/input/titanic-machine-learning-from-disaster/test.csv


In [18]:
import pandas as pd
import numpy as np
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

#Load the dataset
file_path = '/kaggle/input/titanic-machine-learning-from-disaster/train.csv'
titanic_data = pd.read_csv(file_path)

#Display th first few rows of the dataset
titanic_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
# Concatenate all row components into a single text string, excluding the label (Survived)

def concatenate_row(row):
    return ' '.join(str(val) for val in row if val != row['Survived'])

titanic_data['text'] = titanic_data.apply(concatenate_row, axis=1)
titanic_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,text
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,"1 3 Braund, Mr. Owen Harris male 22.0 1 A/5 21..."
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,"2 Cumings, Mrs. John Bradley (Florence Briggs ..."
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,"3 3 Heikkinen, Miss. Laina female 26.0 0 0 STO..."
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,"4 Futrelle, Mrs. Jacques Heath (Lily May Peel)..."
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,"5 3 Allen, Mr. William Henry male 35.0 373450 ..."


In [20]:
# Data Cleaning Cleaning & Feature Engineering (lowercase, punctuation removal, stopwords removal, stem words)

stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def clean_text(text):
    text = text.lower()
    text = ''.join(ch for ch in text if ch not in string.punctuation)
    # Temporarily remove digits so we can convert them later in a separate step
    text = re.sub(r'\d+', '', text)
    words = text.split()
    words = [ps.stem(w) for w in words if w not in stop_words]
    return ' '.join(words)

titanic_data['cleaned_text'] = titanic_data['text'].apply(clean_text)
titanic_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,text,cleaned_text
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,"1 3 Braund, Mr. Owen Harris male 22.0 1 A/5 21...",braund mr owen harri male nan
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,"2 Cumings, Mrs. John Bradley (Florence Briggs ...",cume mr john bradley florenc brigg thayer fema...
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,"3 3 Heikkinen, Miss. Laina female 26.0 0 0 STO...",heikkinen miss laina femal stono nan
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,"4 Futrelle, Mrs. Jacques Heath (Lily May Peel)...",futrel mr jacqu heath lili may peel femal c
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,"5 3 Allen, Mr. William Henry male 35.0 373450 ...",allen mr william henri male nan


In [22]:
# Convert lal numerical digits to text 

# Let's use a dictionary here.

digit_map = {
    '0': 'zero', '1': 'one', '2': 'two', '3': 'three', '4': 'four',
    '5': 'five', '6': 'six', '7': 'seven', '8': 'eight', '9': 'nine'
}

def convert_digits_to_words(text):
    return re.sub(r'\d', lambda x: digit_map[x.group()], text)

titanic_data['digits_to_text'] = titanic_data['text'].apply(convert_digits_to_words)
titanic_data['final_text'] = titanic_data['digits_to_text'].apply(clean_text)
titanic_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,text,cleaned_text,digits_to_text,final_text
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,"1 3 Braund, Mr. Owen Harris male 22.0 1 A/5 21...",braund mr owen harri male nan,"one three Braund, Mr. Owen Harris male twotwo....",one three braund mr owen harri male twotwozero...
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,"2 Cumings, Mrs. John Bradley (Florence Briggs ...",cume mr john bradley florenc brigg thayer fema...,"two Cumings, Mrs. John Bradley (Florence Brigg...",two cume mr john bradley florenc brigg thayer ...
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,"3 3 Heikkinen, Miss. Laina female 26.0 0 0 STO...",heikkinen miss laina femal stono nan,"three three Heikkinen, Miss. Laina female twos...",three three heikkinen miss laina femal twosixz...
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,"4 Futrelle, Mrs. Jacques Heath (Lily May Peel)...",futrel mr jacqu heath lili may peel femal c,"four Futrelle, Mrs. Jacques Heath (Lily May Pe...",four futrel mr jacqu heath lili may peel femal...
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,"5 3 Allen, Mr. William Henry male 35.0 373450 ...",allen mr william henri male nan,"five three Allen, Mr. William Henry male three...",five three allen mr william henri male threefi...


In [23]:
# Now, it's time to build a text classification model

X = titanic_data['final_text']
y = titanic_data['Survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96       105
           1       0.95      0.93      0.94        74

    accuracy                           0.95       179
   macro avg       0.95      0.95      0.95       179
weighted avg       0.95      0.95      0.95       179

